# Assignment 3: Improving the Index

In this assignment, we will improve the search index and query functions from the previous assignment.

## Loading the Data and Defining Auxiliary Functions

This section is copied from the previous notebook.

In [1]:
import pickle, bz2, re
from collections import namedtuple, defaultdict, Counter
from IPython.display import display, HTML
from math import log10, sqrt

Summaries_file = 'data/influenza_Summaries.pkl.bz2'
Abstracts_file = 'data/influenza_Abstracts.pkl.bz2'

Summaries = pickle.load( bz2.BZ2File( Summaries_file, 'rb' ) )
Abstracts = pickle.load( bz2.BZ2File( Abstracts_file, 'rb' ) )

paper = namedtuple( 'paper', ['title', 'authors', 'year', 'doi'] )
for (id, paper_info) in Summaries.items():
    Summaries[id] = paper( *paper_info )

In [2]:
def tokenize(text):
    """
    Function that tokenizes a string in a rather naive way. Can be extended later.
    """
    return text.split(' ')

def preprocess(tokens):
    """
    Perform linguistic preprocessing on a list of tokens. Can be extended later.
    """
    result = []
    for token in tokens:
        result.append(token.lower())
    return result

In [3]:
def display_summary( id, show_abstract=False, show_id=True, extra_text='' ):
    """
    Function for printing a paper's summary through IPython's Rich Display System.
    Trims long author lists, and adds a link to the paper's DOI (when available).
    """
    s = Summaries[id]
    lines = []
    title = s.title
    if s.doi != '':
        title = '<a href=http://dx.doi.org/{:s}>{:s}</a>'.format(s.doi, title)
    title = '<strong>' + title + '</strong>'
    lines.append(title)
    authors = ', '.join( s.authors[:20] ) + ('' if len(s.authors) <= 20 else ', ...')
    lines.append(str(s.year) + '. ' + authors)
    if (show_abstract):
        lines.append('<small><strong>Abstract:</strong> <em>{:s}</em></small>'.format(Abstracts[id]))
    if (show_id):
        lines.append('[ID: {:d}]'.format(id))
    if (extra_text != ''):
         lines.append(extra_text)
    display( HTML('<br>'.join(lines)) )

In [4]:
inverted_index = defaultdict(set)

for (id, abstract) in Abstracts.items():
    for term in preprocess(tokenize(abstract)):
        inverted_index[term].add(id)

## Stemming

As we could see from the results of the last assignment, our simple index doesn't handle punctuation and the difference between singular and plural versions of the same word very well. We won't go much into the details of tokenization and linguistic analysis here, because we also want to focus on scoring and ranking below. Therefore, we are using an existing library for tokenizatoin and stemming, namely the NLTK package. The following line will install NLTK if necessary (or you have to follow [these instructions](http://www.nltk.org/install.html) if that doesn't work):

In [5]:
! pip install --user nltk

In [6]:
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import EnglishStemmer
import nltk
nltk.download('punkt')
stemmer = EnglishStemmer()

s = '''Good muffins cost $3.88\nin New York.  Please buy me two of them.\n\nThanks.'''

print('INPUT TEXT:\n ', s)

print('TOKENIZE: ', tokenize(s))
print('WORD TOKENIZE: ', word_tokenize(s))

[nltk_data] Downloading package punkt to /home/tk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
INPUT TEXT:
  Good muffins cost $3.88
in New York.  Please buy me two of them.

Thanks.
TOKENIZE:  ['Good', 'muffins', 'cost', '$3.88\nin', 'New', 'York.', '', 'Please', 'buy', 'me', 'two', 'of', 'them.\n\nThanks.']
WORD TOKENIZE:  ['Good', 'muffins', 'cost', '$', '3.88', 'in', 'New', 'York', '.', 'Please', 'buy', 'me', 'two', 'of', 'them', '.', 'Thanks', '.']


In [7]:
print(stemmer.stem("processes"))

process


## Ranking

Another important method to improve our search results is to rank them, which can be done by calculating a score for each document based on the matching terms from the query. One such scoring method is *tf-idf*, which comes with several variants, as explained in the lecture slides.

In order to quickly calculate the scores for a term/document combination, we'll need quick access to a couple of things:

- tf(t,d): How often does a term occur in a document
- df(t): In how many documents does a term occur
- num_documents: The number of documents in our index

In [8]:
tf_matrix = defaultdict(Counter)

for (doc_id, abstract) in Abstracts.items():
    tokens = preprocess(tokenize(abstract))
    tf_matrix[doc_id] = Counter(tokens)

def tf(t,d):
    return float(tf_matrix[d][t])

def df(t):
    return float(len(inverted_index[t]))

num_documents = float(len(Abstracts))

Let's test these functions with some examples:

In [9]:
print(tf('madagascar', 12458917))
print(df('madagascar'))
print(num_documents)

6.0
11.0
87602.0


With these helper functions, we can now easily calculate the _tf-idf_ weights of a term in a document by implementing the weighting formula from the slides, which you will do in the assignments below.

----------

# Tasks

**Your name:** ...

### Task 1

Implement in the code block below the `smarter_tokenize` function using NLTK's function for tokenization, and the `smarter_preprocess` function to perform stemming in addition to case normalization. 

In [10]:
# Smarter linguistic processing

def smarter_tokenize(text):  
    return word_tokenize(text)  

def smarter_preprocess(tokens):  
    result = []  
    for token in tokens:  
        result.append(stemmer.stem(token))  
    return result  

# To test it:
print(smarter_preprocess(smarter_tokenize("He buys many books, some about I.R., for less than $1.50!")))

['he', 'buy', 'mani', 'book', ',', 'some', 'about', 'i.r.', ',', 'for', 'less', 'than', '$', '1.50', '!']


Now we can make a smarter index based on these functions. For practical purposes, the code below generates the smarter index on a subset of the data, as generating an index with stemming on the entire set would take too much time. (You don't need to change or add anything in the code block below. Just leave it as it is.)

In [11]:
# Below, we create our smarter index (based on a subset of the documents for demonstration purposes)
smarter_index = defaultdict(set)

# Here we define the subset (somewhat arbitrary):
subset_of_ids = set(key for key in Abstracts.keys() if 28900000 <= key < 29000000)
subset_of_abstracts = ((key, Abstracts[key]) for key in subset_of_ids)

# Building our smarter index:
for (id, abstract) in subset_of_abstracts:
    for term in smarter_preprocess(smarter_tokenize(abstract)):
        smarter_index[term].add(id)

Now implement the `smarter_and_query` function, based on the two functions `smarter_tokenize` and `smarter_preprocess` you defined above and accessing our new index `smarter_index`. You can start from the code for `and_query` from the last assignment.

In [12]:
# Smarter and_query based on the smarter tokenize and preprocess functions

def smarter_and_query(query_string):  
    query_words = smarter_preprocess(smarter_tokenize(query_string))  
    first_word = query_words[0]  
    remaining_words = query_words[1:]  
    intersection_set = smarter_index[first_word]  
    for t in remaining_words:  
        intersection_set = intersection_set.intersection(smarter_index[t])  
    return intersection_set  

### Task 2

Run the queries "therapeutic protections antibodies" and "investigate de novo protein design" with the new `smarter_and_query` function from task 1. Do they return paper *28953867* (this is our exemplary paper from the last assignment)? For each of the two example queries, what do our new smarter functions specifically contribute to the result (as compared to our previous naive implementations for tokenization and preprocessing)?

In [13]:
smarter_and_query('therapeutic protections antibodies')  

{28953867, 28966056}

In [14]:
smarter_and_query('investigate de novo protein design')  

{28953867}

In [15]:
def and_query(query_string):  
    query_words = preprocess(tokenize(query_string))  
    first_word = query_words[0]  
    remaining_words = query_words[1:]  
    intersection_set = inverted_index[first_word]  
    for t in remaining_words:  
        intersection_set = intersection_set.intersection(inverted_index[t])  
    return intersection_set  

In [16]:
and_query('investigated, de novo protein design')  

{28953867}

**Answer:** Our smarter tokenization makes sure that the word "antibodies", which occurs in the context "... unlike antibodies, do not ... " in the abstract of the given article, is mapped to `antibodies`, without the comma being attached to it. The smarter stemming moreover maps "protection" and "protections" both to the same stemmed term "protect". Otherwise our first query above would not have returned the document. Both methods, tokenization and stemming, are required for the term `investigate` in the second query to be correctly mapped, as it occurs in the context "... previously investigated, enabled ..." and would otherwise have a comma attached and be recognized as unstemmed "investigated".

### Task 3

Now we move to a different subject and use our old index again. That is, we **don't** use the smarter functions defined above for tasks 3 to 5!

Create a function `tfidf(t,d)` that returns the tf-idf score of term `t` in document `d` by using `tf(t,d)`, `df(t)` and `num_documents` as defined above. To do this, first implement a function `idf(t)` to calculate the inverse document frequency, and then use this function to calculate the full tf-idf. The tf-idf formula can be found on the lecture slides. Use tf-idf with plain (non-logarithmic) term frequency, as applied by scoring variant `ntn`. Test your function with the examples shown below. You can use the `log10(n)` function to calculate the base 10 logarithm.

Again, use our old (non-smart) index for this task and the tasks below, and **not** the functions defined in tasks 1 and 2.

In [17]:
def idf(t):  
    return log10(num_documents/df(t))  

def tfidf(t,d):  
    return tf(t,d) * idf(t)  

print(tfidf('botulinum', 28953867))
print(tfidf('protection', 28953867))
print(tfidf('with', 28953867))

3.9882715120146983
1.3759586905709682
0.5709737348367643


### Task 4

Create a function `query_ntn(query_string)`, which accepts as input a single query string of one or more words, and returns or prints a list of (up to) 10 best matching documents, along with their score. Use _tf-idf_ to calculate document scores based on the query, applying variant `ntn`, as above (see the formula for the `ntn` version of scoring on the lecture slides). Use an auxiliary function `score_ntn` to calculate the score. The results should be shown in descending order by score.

You can start by copying your `or_query` function from assignment 2, then expand that to rank the results, making use of the `tfidf(t,d)` function you created above.

Demonstrate your function by giving it the exemplary query string "adverse effects of influenza vaccination".

In [18]:
def score_ntn(query_words, doc_id):  
    score = 0  
    for t in query_words:  
        score += tfidf(t,doc_id)  
    return score  

def query_ntn(query_string):  
    query_words = preprocess(tokenize(query_string))  
    first_word = query_words[0]  
    remaining_words = query_words[1:]  
    union_set = inverted_index[first_word]  
    for t in remaining_words:  
        union_set = union_set.union(inverted_index[t])  
    for paper_id in sorted(union_set, key=lambda i: score_ntn(query_words,i), reverse=True)[:10]:  
        paper = Summaries[paper_id]  
        print(str(paper_id) + ' ' + paper.title + ' (score: ' + str(score_ntn(query_words,paper_id)) + ')')  

# Example query:
query_ntn("adverse effects of influenza vaccination")

16548114 Antiviral drugs in influenza: an adjunct to vaccination in some situations. (score: 22.345895627761813)
22399172 [Attitudes and side effects related to pandemic influenza A (H1N1) vaccination in healthcare personnel]. (score: 19.796925845349847)
29058218 Ontology-Based Vaccine Adverse Event Representation and Analysis. (score: 19.582531918161074)
27381724 Learning to Trust Flu Shots: Quasi-Experimental Evidence from the 2009 Swine Flu Pandemic. (score: 19.180087421268432)
23209624 Ontology-based combinatorial comparative analysis of adverse events associated with killed and live influenza vaccines. (score: 18.803238995205064)
26319740 Does influenza vaccination improve pregnancy outcome? Methodological issues and research needs. (score: 18.775242000599004)
17957995 [Adverse events following influenza vaccination: reaction to specific reports and the necessity of a central registration system]. (score: 18.48975089157836)
29514719 Protecting Healthcare Personnel in Outpatient Se

### Task 5

In this last task, you should create a second version of the query function from Task 4, called `query_ntc`. This second version should use, as its name suggests, variant `ntc` instead of `ntn`, and therefore apply the cosine similarity measure, in addition to applying _tf-idf_. For this, consult the formula for variant `nnc` on the lecture slides and adopt it to include the _idf_ metric (that is, add the `t` element of `ntc`). (You can drop the square root of |q| in the formula, as indicated on the slides.)

As a first step, we can calculate beforehand the length of all document vectors (because they don't depend on the query) for document vectors consisting of _tf-idf_ values. The code below does just that, assuming that you defined the function `tfidf(t,d)` above (don't change this code block, just run it):

In [19]:
tfidf_length_values = defaultdict(int)

for (doc_id, abstract) in Abstracts.items():
    l = 0
    for t in tf_matrix[doc_id].keys():
        l += tfidf(t,doc_id) ** 2
    tfidf_length_values[doc_id] = sqrt(l)

def tfidf_length(d):
    return tfidf_length_values[d]

We can now get the length of a document vector by calling `tfidf_length(d)`.

Based on this, you can now implement `query_ntc` in the code block below. You should again first define an auxiliary function, called `score_ntc`. You can start by copy-pasting the code from Task 4.

To output the results, use the provided `display_summary` function to make the output a bit more like the results page of a search engine. Lastly, demonstrate your `query_ntc` function with the same example query as above: "adverse effects of influenza vaccination"

In [20]:
def score_ntc(query_words, doc_id):  
    score = 0  
    for t in query_words:  
        query_term_weight = 1 * idf(t)  
        score += tfidf(t, doc_id) * query_term_weight  
    return score / tfidf_length(doc_id)  

def query_ntc(query_string):  
    query_words = preprocess(tokenize(query_string))  
    first_word = query_words[0]  
    remaining_words = query_words[1:]  
    union_set = inverted_index[first_word]  
    for t in remaining_words:  
        union_set = union_set.union(inverted_index[t])  
    for paper_id in sorted(union_set, key=lambda i: score_ntc(query_words,i), reverse=True)[:10]:  
        paper = Summaries[paper_id]  
        display_summary(paper_id, extra_text='Score: ' + str(score_ntc(query_words,paper_id))) 

# Example query:
query_ntc("adverse effects of influenza vaccination")  

# Submission

Submit the answers to the assignment via Canvas as a modified version of this Notebook file (file with `.ipynb` extension) that includes your code and your answers.

Before submitting, restart the kernel and re-run the complete code (**Kernel > Restart & Run All**), and then check whether your assignment code still works as expected.

Don't forget to add your name, and remember that the assignments have to be done individually and group submissions are **not allowed**.